In [2]:
"""
README: DeepLabV3 Semantic Segmentation Per-Class, Per-City Best-10 Visualizer

This script:
- Runs inference on a directory of images with a trained DeepLabV3+MobileNetV3 segmentation model,
- Computes IoU (Intersection over Union) for each class, per city,
- Saves the best-10 (highest IoU) segmentation results per class/city as visualization sets:
    - original.jpg (original image)
    - mask.jpg     (ground truth mask overlay, always green)
    - prediction.jpg (predicted mask overlay, class-specific color)
- Output folders are organized by class and city for easy visual benchmarking.

How to use:
- Set the paths for your trained model, test images, and output directory below.
- Optionally, edit the class and city definitions as needed.
- Run the script; visual outputs will be created in OUTPUT_DIR.

Requirements:
- Python 3.8+
- torch
- torchvision
- opencv-python
- numpy
- tqdm

Author: Bahadir Akin Akgul
Date: 13.07.2025
"""

import os
import cv2
import torch
import numpy as np
from tqdm import tqdm
from pathlib import Path
from collections import defaultdict
import torchvision.transforms as T

# === SETTINGS ===
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = "PATH_TO_TRAINED_MODEL/trained_model.pth"
DATA_DIR = "PATH_TO_TEST_IMAGES"
OUTPUT_DIR = Path("deeplab-best-10-segmented")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Class IDs (model output indices)
CLASS_IDS = {
    'road': 1,
    'pedestrian': 2,
    'vehicle': 3
}

# Prediction colors (prediction.jpg only)
CLASS_COLORS = {
    1: (255, 0, 255),     # road - magenta
    2: (0, 255, 255),     # pedestrian - yellow
    3: (0, 0, 255)        # vehicle - red
}

MASK_COLOR = (0, 255, 0)  # green – for all mask.jpg

city_keywords = {
    'istanbul': ['libadiye', 'levent', 'taksim', 'ciragan', 'barbaros', 'dolmabahce', 'bagdat', 'muallim', 'katar'],
    'paris': ['paris-champs'],
    'munich': ['munih'],
    'marseille': ['marsilya']
}
city_translation = {
    'istanbul': 'Istanbul',
    'paris': 'Paris',
    'munich': 'Munich',
    'marseille': 'Marseille',
    'unknown': 'Unknown'
}

def get_city(filename):
    for city, keywords in city_keywords.items():
        if any(k in filename for k in keywords):
            return city
    return 'unknown'

def overlay_mask(image, mask, cls_id, color, alpha=0.5):
    overlay = image.copy()
    region = (mask == cls_id)
    color_layer = np.zeros_like(image, dtype=np.uint8)
    color_layer[region] = color
    return cv2.addWeighted(color_layer, alpha, overlay, 1 - alpha, 0)

# === LOAD MODEL ===
import torchvision.models.segmentation as models
model = models.deeplabv3_mobilenet_v3_large(weights=None, num_classes=4)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval().to(DEVICE)

# === DATASET ===
image_files = sorted([f for f in os.listdir(DATA_DIR) if f.endswith(".jpg")])
transform = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

results_per_class = {k: defaultdict(list) for k in CLASS_IDS}

# === INFERENCE ===
print("Inference started...")
for fname in tqdm(image_files):
    name = os.path.splitext(fname)[0]
    img_path = os.path.join(DATA_DIR, fname)
    mask_path = os.path.join(DATA_DIR, name + "_mask.png")

    image = cv2.imread(img_path)
    if image is None:
        continue
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None or image.shape[:2] != mask.shape:
        continue

    input_tensor = transform(image).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        pred = model(input_tensor)['out'].squeeze().argmax(0).cpu().numpy()

    for label, cls_id in CLASS_IDS.items():
        pred_mask = (pred == cls_id).astype(np.uint8)
        gt_mask = (mask == cls_id).astype(np.uint8)

        inter = np.logical_and(pred_mask, gt_mask).sum()
        union = np.logical_or(pred_mask, gt_mask).sum()
        iou = inter / (union + 1e-6)

        if inter > 0:
            city = get_city(name)
            results_per_class[label][city].append({
                "iou": iou,
                "image": image.copy(),
                "gt": mask.copy(),
                "pred": pred.copy(),
                "name": name
            })

print("Inference completed.")

# === SAVE TOP 10 RESULTS ===
for label, cls_id in CLASS_IDS.items():
    for city, records in results_per_class[label].items():
        top10 = sorted(records, key=lambda x: -x['iou'])[:10]
        city_name = city_translation.get(city, city)
        save_dir = OUTPUT_DIR / label / city_name
        save_dir.mkdir(parents=True, exist_ok=True)

        for i, r in enumerate(top10, 1):
            name = f"{i:02d}_{r['name']}"
            dst = save_dir / name
            dst.mkdir(parents=True, exist_ok=True)

            img = r['image']
            gt_masked = overlay_mask(img, r['gt'], cls_id, MASK_COLOR)
            pred_masked = overlay_mask(img, r['pred'], cls_id, CLASS_COLORS[cls_id])

            cv2.imwrite(str(dst / "original.jpg"), img)
            cv2.imwrite(str(dst / "mask.jpg"), gt_masked)
            cv2.imwrite(str(dst / "prediction.jpg"), pred_masked)

            print(f"Saved: {label}/{city_name}/{name}")

print("All visualizations saved.")


🔍 İnferencing başlıyor...


100%|██████████| 1316/1316 [02:22<00:00,  9.23it/s]


✅ İnferencing tamamlandı.
📂 Kaydedildi: road/Istanbul/01_muallimnaci-286_jpg.rf.0fdc57447155fcb982daae2e0539cd26
📂 Kaydedildi: road/Istanbul/02_bagdat-1-24_jpg.rf.72676e72054cf6522faf8369ba6bbced
📂 Kaydedildi: road/Istanbul/03_bagdat-1-342_jpg.rf.e377aa9fe6192b90ded430a02ee67c64
📂 Kaydedildi: road/Istanbul/04_bagdat-1-124_jpg.rf.96ffc5454f54898428232bfb72c8bdb4
📂 Kaydedildi: road/Istanbul/05_libadiye-318_jpg.rf.4a3330313065e0bced7acc55c066ec79
📂 Kaydedildi: road/Istanbul/06_bagdat-1-387_jpg.rf.e56c8ae432e48c6ad3ab1c8ec4879c61
📂 Kaydedildi: road/Istanbul/07_taksim-100_jpg.rf.be60ff28fe4d26482a6e2faebc740bfb
📂 Kaydedildi: road/Istanbul/08_bagdat-1-318_jpg.rf.beb44ee2457786f154172f6e2c5d0770
📂 Kaydedildi: road/Istanbul/09_muallimnaci-1038_jpg.rf.310083c7558ce84d0e315011a4e317b8
📂 Kaydedildi: road/Istanbul/10_libadiye-539_jpg.rf.72dd10809b7772d1a0a7df22e3bc90ef
📂 Kaydedildi: road/Marseille/01_marsilya2-564_jpg.rf.c5b3035f8b914bd589655a2ae9fdffb4
📂 Kaydedildi: road/Marseille/02_marsilya-184

In [1]:
import os
import cv2
import torch
import numpy as np
from tqdm import tqdm
from pathlib import Path
from collections import defaultdict
import torchvision.transforms as T

# === AYARLAR ===
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = "road-tr-od-ss-70-30-results/trained_model.pth"
DATA_DIR = "yolo-seg-11042025/road-tr-od-ss/test"
OUTPUT_DIR = Path("deeplab-best-10-segmented")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# === SINIFLAR ===
CLASS_IDS = {
    'road': 1,
    'pedestrian': 2,
    'vehicle': 3
}
CLASS_COLORS = {
    1: (255, 0, 255),     # road - magenta
    2: (0, 255, 255),     # pedestrian - yellow
    3: (0, 0, 255)        # vehicle - red
}

city_keywords = {
    'istanbul': ['libadiye', 'levent', 'taksim', 'ciragan', 'barbaros', 'dolmabahce', 'bagdat', 'muallim', 'katar'],
    'paris': ['paris-champs'],
    'munih': ['munih'],
    'marsilya': ['marsilya']
}
city_translation = {
    'istanbul': 'Istanbul',
    'paris': 'Paris',
    'munih': 'Munich',
    'marsilya': 'Marseille',
    'unknown': 'Unknown'
}

def get_city(filename):
    for city, keywords in city_keywords.items():
        if any(k in filename for k in keywords):
            return city
    return 'unknown'

def overlay_mask(image, mask, cls_id, color, alpha=0.5):
    overlay = image.copy()
    region = (mask == cls_id)
    color_layer = np.zeros_like(image, dtype=np.uint8)
    color_layer[region] = color
    return cv2.addWeighted(color_layer, alpha, overlay, 1 - alpha, 0)

# === MODEL YÜKLE ===
import torchvision.models.segmentation as models
model = models.deeplabv3_mobilenet_v3_large(weights=None, num_classes=4)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval().to(DEVICE)

# === VERİ SETİ ===
image_files = sorted([f for f in os.listdir(DATA_DIR) if f.endswith(".jpg")])
transform = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

results_per_class = {k: defaultdict(list) for k in CLASS_IDS}

# === INFERENCING ===
print("🔍 İnferencing başlıyor...")
for fname in tqdm(image_files):
    name = os.path.splitext(fname)[0]
    img_path = os.path.join(DATA_DIR, fname)
    mask_path = os.path.join(DATA_DIR, name + "_mask.png")

    image = cv2.imread(img_path)
    if image is None:
        continue
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None or image.shape[:2] != mask.shape:
        continue

    input_tensor = transform(image).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        pred = model(input_tensor)['out'].squeeze().argmax(0).cpu().numpy()

    # === METRİK HESAPLAMA ===
    for label, cls_id in CLASS_IDS.items():
        pred_mask = (pred == cls_id).astype(np.uint8)
        gt_mask = (mask == cls_id).astype(np.uint8)

        inter = np.logical_and(pred_mask, gt_mask).sum()
        union = np.logical_or(pred_mask, gt_mask).sum()
        iou = inter / (union + 1e-6)

        if inter > 0:
            city = get_city(name)
            results_per_class[label][city].append({
                "iou": iou,
                "image": image.copy(),
                "gt": mask.copy(),
                "pred": pred.copy(),
                "name": name
            })

print("✅ İnferencing tamamlandı.")

# === EN İYİ 10 KAYDET ===
for label, cls_id in CLASS_IDS.items():
    for city, records in results_per_class[label].items():
        top10 = sorted(records, key=lambda x: -x['iou'])[:10]
        city_name = city_translation.get(city, city)
        save_dir = OUTPUT_DIR / label / city_name
        save_dir.mkdir(parents=True, exist_ok=True)

        for i, r in enumerate(top10, 1):
            name = f"{i:02d}_{r['name']}"
            dst = save_dir / name
            dst.mkdir(parents=True, exist_ok=True)

            # Maskeleri renkle kapla
            img = r['image']
            gt_masked = overlay_mask(img, r['gt'], cls_id, CLASS_COLORS[cls_id])
            pred_masked = overlay_mask(img, r['pred'], cls_id, CLASS_COLORS[cls_id])

            cv2.imwrite(str(dst / "original.jpg"), img)
            cv2.imwrite(str(dst / "mask.jpg"), gt_masked)
            cv2.imwrite(str(dst / "prediction.jpg"), pred_masked)

            print(f"📂 Kaydedildi: {label}/{city_name}/{name}")

print("🎉 Tüm görseller kaydedildi.")


🔍 İnferencing başlıyor...


100%|██████████| 1316/1316 [02:38<00:00,  8.29it/s]


✅ İnferencing tamamlandı.
📂 Kaydedildi: road/Istanbul/01_muallimnaci-286_jpg.rf.0fdc57447155fcb982daae2e0539cd26
📂 Kaydedildi: road/Istanbul/02_bagdat-1-24_jpg.rf.72676e72054cf6522faf8369ba6bbced
📂 Kaydedildi: road/Istanbul/03_bagdat-1-342_jpg.rf.e377aa9fe6192b90ded430a02ee67c64
📂 Kaydedildi: road/Istanbul/04_bagdat-1-124_jpg.rf.96ffc5454f54898428232bfb72c8bdb4
📂 Kaydedildi: road/Istanbul/05_libadiye-318_jpg.rf.4a3330313065e0bced7acc55c066ec79
📂 Kaydedildi: road/Istanbul/06_bagdat-1-387_jpg.rf.e56c8ae432e48c6ad3ab1c8ec4879c61
📂 Kaydedildi: road/Istanbul/07_taksim-100_jpg.rf.be60ff28fe4d26482a6e2faebc740bfb
📂 Kaydedildi: road/Istanbul/08_bagdat-1-318_jpg.rf.beb44ee2457786f154172f6e2c5d0770
📂 Kaydedildi: road/Istanbul/09_muallimnaci-1038_jpg.rf.310083c7558ce84d0e315011a4e317b8
📂 Kaydedildi: road/Istanbul/10_libadiye-539_jpg.rf.72dd10809b7772d1a0a7df22e3bc90ef
📂 Kaydedildi: road/Marseille/01_marsilya2-564_jpg.rf.c5b3035f8b914bd589655a2ae9fdffb4
📂 Kaydedildi: road/Marseille/02_marsilya-184

In [1]:
import os
import cv2
import torch
import numpy as np
from tqdm import tqdm
from pathlib import Path
from collections import defaultdict
import torchvision.transforms as T
from PIL import Image  # 👈 300 DPI için eklendi

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = "road-tr-od-ss-70-30-results/trained_model.pth"
DATA_DIR = "yolo-seg-11042025/road-tr-od-ss/test"
OUTPUT_DIR = Path("deeplab-best-10-segmented-300dpi")  # 👈 GÜNCELLENDİ
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Class IDs (model output indices)
CLASS_IDS = {
    'road': 1,
    'pedestrian': 2,
    'vehicle': 3
}

CLASS_COLORS = {
    1: (255, 0, 255),     # road - magenta
    2: (0, 255, 255),     # pedestrian - yellow
    3: (0, 0, 255)        # vehicle - red
}

MASK_COLOR = (0, 255, 0)  # green – GT maskeleri için

city_keywords = {
    'istanbul': ['libadiye', 'levent', 'taksim', 'ciragan', 'barbaros', 'dolmabahce', 'bagdat', 'muallim', 'katar'],
    'paris': ['paris-champs'],
    'munih': ['munih'],
    'marsilya': ['marsilya']
}
city_translation = {
    'istanbul': 'Istanbul',
    'paris': 'Paris',
    'munih': 'Munich',
    'marsilya': 'Marseille',
    'unknown': 'Unknown'
}

def get_city(filename):
    for city, keywords in city_keywords.items():
        if any(k in filename for k in keywords):
            return city
    return 'unknown'

def overlay_mask(image, mask, cls_id, color, alpha=0.5):
    overlay = image.copy()
    region = (mask == cls_id)
    color_layer = np.zeros_like(image, dtype=np.uint8)
    color_layer[region] = color
    return cv2.addWeighted(color_layer, alpha, overlay, 1 - alpha, 0)

def save_with_dpi(img_array, path, dpi=(300, 300)):
    img_rgb = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img_rgb)
    img_pil.save(path, dpi=dpi, quality=95)

# === MODEL YÜKLE ===
import torchvision.models.segmentation as models
model = models.deeplabv3_mobilenet_v3_large(weights=None, num_classes=4)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval().to(DEVICE)

# === VERİ SETİ ===
image_files = sorted([f for f in os.listdir(DATA_DIR) if f.endswith(".jpg")])
transform = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

results_per_class = {k: defaultdict(list) for k in CLASS_IDS}

# === INFERENCING ===
print("🔍 İnferencing başlıyor...")
for fname in tqdm(image_files):
    name = os.path.splitext(fname)[0]
    img_path = os.path.join(DATA_DIR, fname)
    mask_path = os.path.join(DATA_DIR, name + "_mask.png")

    image = cv2.imread(img_path)
    if image is None:
        continue
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None or image.shape[:2] != mask.shape:
        continue

    input_tensor = transform(image).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        pred = model(input_tensor)['out'].squeeze().argmax(0).cpu().numpy()

    for label, cls_id in CLASS_IDS.items():
        pred_mask = (pred == cls_id).astype(np.uint8)
        gt_mask = (mask == cls_id).astype(np.uint8)

        inter = np.logical_and(pred_mask, gt_mask).sum()
        union = np.logical_or(pred_mask, gt_mask).sum()
        iou = inter / (union + 1e-6)

        if inter > 0:
            city = get_city(name)
            results_per_class[label][city].append({
                "iou": iou,
                "image": image.copy(),
                "gt": mask.copy(),
                "pred": pred.copy(),
                "name": name
            })

print("✅ İnferencing tamamlandı.")

# === EN İYİ 10 KAYDET ===
for label, cls_id in CLASS_IDS.items():
    for city, records in results_per_class[label].items():
        top10 = sorted(records, key=lambda x: -x['iou'])[:10]
        city_name = city_translation.get(city, city)
        save_dir = OUTPUT_DIR / label / city_name
        save_dir.mkdir(parents=True, exist_ok=True)

        for i, r in enumerate(top10, 1):
            name = f"{i:02d}_{r['name']}"
            dst = save_dir / name
            dst.mkdir(parents=True, exist_ok=True)

            img = r['image']
            gt_masked = overlay_mask(img, r['gt'], cls_id, MASK_COLOR)
            pred_masked = overlay_mask(img, r['pred'], cls_id, CLASS_COLORS[cls_id])

            save_with_dpi(img, dst / "original.jpg")
            save_with_dpi(gt_masked, dst / "mask.jpg")
            save_with_dpi(pred_masked, dst / "prediction.jpg")

            print(f"📂 Kaydedildi: {label}/{city_name}/{name}")

print("🎉 Tüm görseller 300 DPI ile kaydedildi.")


🔍 İnferencing başlıyor...


100%|██████████| 1316/1316 [26:56<00:00,  1.23s/it]  


✅ İnferencing tamamlandı.
📂 Kaydedildi: road/Istanbul/01_muallimnaci-286_jpg.rf.0fdc57447155fcb982daae2e0539cd26
📂 Kaydedildi: road/Istanbul/02_bagdat-1-24_jpg.rf.72676e72054cf6522faf8369ba6bbced
📂 Kaydedildi: road/Istanbul/03_bagdat-1-342_jpg.rf.e377aa9fe6192b90ded430a02ee67c64
📂 Kaydedildi: road/Istanbul/04_bagdat-1-124_jpg.rf.96ffc5454f54898428232bfb72c8bdb4
📂 Kaydedildi: road/Istanbul/05_libadiye-318_jpg.rf.4a3330313065e0bced7acc55c066ec79
📂 Kaydedildi: road/Istanbul/06_bagdat-1-387_jpg.rf.e56c8ae432e48c6ad3ab1c8ec4879c61
📂 Kaydedildi: road/Istanbul/07_taksim-100_jpg.rf.be60ff28fe4d26482a6e2faebc740bfb
📂 Kaydedildi: road/Istanbul/08_bagdat-1-318_jpg.rf.beb44ee2457786f154172f6e2c5d0770
📂 Kaydedildi: road/Istanbul/09_muallimnaci-1038_jpg.rf.310083c7558ce84d0e315011a4e317b8
📂 Kaydedildi: road/Istanbul/10_libadiye-539_jpg.rf.72dd10809b7772d1a0a7df22e3bc90ef
📂 Kaydedildi: road/Marseille/01_marsilya2-564_jpg.rf.c5b3035f8b914bd589655a2ae9fdffb4
📂 Kaydedildi: road/Marseille/02_marsilya-184